<style>
  .custom-title {
    font-size: 48px;
    font-weight: bold;
    color: #007acc; /* You can change the color to your preference */
    margin-bottom: 10px;
  }
</style>

<div class="custom-title">Messing around with Local NLP</div>


In this notebook I was reviewing the local code I had prior to working with Cloud NLP. This is just a dump of what you could do locally, and see the limitations, caveats, and use cases.

## Imports

In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

from textblob import TextBlob


In [2]:
sekiro = pd.read_csv("games/sekiro_df.csv")
civ6 = pd.read_csv("games/civ6_df.csv")
nba = pd.read_csv("games/nba_df.csv")
gtav = pd.read_csv("games/gtav_df.csv")
phasmo = pd.read_csv("games/phasmo_df.csv")
witcher = pd.read_csv("games/witcher_df.csv")

In [3]:
# Drop the first two columns from each DataFrame
dfs = [sekiro, civ6, nba, gtav, phasmo, witcher]
for df in dfs:
    df.drop(df.columns[:3], axis=1, inplace=True)

In [4]:
sekiro

,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,Sekiro™: Shadows Die Twice,57953979,english,Still can't beat the Blazing Bull. 10/10,1574800184,1574800184,True,1,1,0.520000,...,True,False,False,76561198102339835,2313,23,498718.0,0.0,310344.0,1.590117e+09
1,Sekiro™: Shadows Die Twice,60553783,english,"It's okay, I guess.",1577312423,1577312423,True,10,32,0.572634,...,True,False,False,76561198846798934,2159,55,349688.0,0.0,175939.0,1.588410e+09
2,Sekiro™: Shadows Die Twice,59813066,english,not bad.,1576305629,1585450858,True,0,2,0.000000,...,True,False,False,76561198329996506,149,5,305363.0,0.0,305060.0,1.606347e+09
3,Sekiro™: Shadows Die Twice,77197015,english,this is a game about fighting through a supern...,1602102651,1602102651,True,333,19,0.901754,...,True,False,False,76561197962008935,278,37,223144.0,0.0,176636.0,1.605052e+09
4,Sekiro™: Shadows Die Twice,49702419,english,Yes,1553291428,1560990683,True,2,0,0.545455,...,False,False,False,76561198073534796,329,23,113698.0,0.0,2916.0,1.579496e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Sekiro™: Shadows Die Twice,78007778,english,Best game uvr,1603444767,1603444767,True,0,0,0.000000,...,True,False,False,76561198812769444,50,4,14898.0,53.0,8992.0,1.610004e+09
496,Sekiro™: Shadows Die Twice,54494063,english,"It's good and everyone knows it's good, an 8/1...",1565142997,1565142997,True,1,0,0.528986,...,True,False,False,76561198042252639,139,20,14887.0,0.0,10227.0,1.604755e+09
497,Sekiro™: Shadows Die Twice,56212472,english,moui epico,1572653706,1572653706,True,0,0,0.000000,...,True,False,False,76561198325611644,77,7,14884.0,122.0,2212.0,1.609718e+09
498,Sekiro™: Shadows Die Twice,59311256,english,This gameplay makes darksouls difficulty feel ...,1575588311,1575588311,True,0,1,0.000000,...,True,False,False,76561198150429552,79,15,14882.0,0.0,1043.0,1.605605e+09


## About the Data

In [5]:
to_keep = ['app_name', 'language','review']

In [6]:
def view_game(df):
    display(df.loc[:, to_keep])

In [7]:
view_game(sekiro)

,app_name,language,review
0,Sekiro™: Shadows Die Twice,english,Still can't beat the Blazing Bull. 10/10
1,Sekiro™: Shadows Die Twice,english,"It's okay, I guess."
2,Sekiro™: Shadows Die Twice,english,not bad.
3,Sekiro™: Shadows Die Twice,english,this is a game about fighting through a supern...
4,Sekiro™: Shadows Die Twice,english,Yes
...,...,...,...
495,Sekiro™: Shadows Die Twice,english,Best game uvr
496,Sekiro™: Shadows Die Twice,english,"It's good and everyone knows it's good, an 8/1..."
497,Sekiro™: Shadows Die Twice,english,moui epico
498,Sekiro™: Shadows Die Twice,english,This gameplay makes darksouls difficulty feel ...


## Local NLP Using VADER

### No Preprocessing

In [8]:
def add_sentiment_column(df, column):
    if column not in df.columns:
        raise ValueError("DataFrame must have a 'review' column for sentiment analysis.")
    
    # Initialize the sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()
    
    # Apply sentiment analysis and create a new 'sentiment' column
    df['sentiment'] = df[column].apply(lambda review: analyzer.polarity_scores(review)['compound'])
    
    return df

In [391]:
def add_rating_column(df, column):
    df[column] = df['sentiment'].apply(lambda sentiment: 
                                         'positive' if sentiment > 0 
                                         else 'negative' if sentiment < 0
                                         else 'neutral')
    return df

In [392]:
add_sentiment_column(sekiro, 'review')

,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,sentiment
0,Sekiro™: Shadows Die Twice,57953979,english,Still can't beat the Blazing Bull. 10/10,1574800184,1574800184,True,1,1,0.520000,...,False,False,76561198102339835,2313,23,498718.0,0.0,310344.0,1.590117e+09,0.0000
1,Sekiro™: Shadows Die Twice,60553783,english,"It's okay, I guess.",1577312423,1577312423,True,10,32,0.572634,...,False,False,76561198846798934,2159,55,349688.0,0.0,175939.0,1.588410e+09,0.2263
2,Sekiro™: Shadows Die Twice,59813066,english,not bad.,1576305629,1585450858,True,0,2,0.000000,...,False,False,76561198329996506,149,5,305363.0,0.0,305060.0,1.606347e+09,0.4310
3,Sekiro™: Shadows Die Twice,77197015,english,this is a game about fighting through a supern...,1602102651,1602102651,True,333,19,0.901754,...,False,False,76561197962008935,278,37,223144.0,0.0,176636.0,1.605052e+09,0.9863
4,Sekiro™: Shadows Die Twice,49702419,english,Yes,1553291428,1560990683,True,2,0,0.545455,...,False,False,76561198073534796,329,23,113698.0,0.0,2916.0,1.579496e+09,0.4019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Sekiro™: Shadows Die Twice,78007778,english,Best game uvr,1603444767,1603444767,True,0,0,0.000000,...,False,False,76561198812769444,50,4,14898.0,53.0,8992.0,1.610004e+09,0.6369
496,Sekiro™: Shadows Die Twice,54494063,english,"It's good and everyone knows it's good, an 8/1...",1565142997,1565142997,True,1,0,0.528986,...,False,False,76561198042252639,139,20,14887.0,0.0,10227.0,1.604755e+09,-0.3489
497,Sekiro™: Shadows Die Twice,56212472,english,moui epico,1572653706,1572653706,True,0,0,0.000000,...,False,False,76561198325611644,77,7,14884.0,122.0,2212.0,1.609718e+09,0.0000
498,Sekiro™: Shadows Die Twice,59311256,english,This gameplay makes darksouls difficulty feel ...,1575588311,1575588311,True,0,1,0.000000,...,False,False,76561198150429552,79,15,14882.0,0.0,1043.0,1.605605e+09,-0.3470


In [393]:
add_rating_column(sekiro, 'rating')

,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,sentiment,rating
0,Sekiro™: Shadows Die Twice,57953979,english,Still can't beat the Blazing Bull. 10/10,1574800184,1574800184,True,1,1,0.520000,...,False,76561198102339835,2313,23,498718.0,0.0,310344.0,1.590117e+09,0.0000,neutral
1,Sekiro™: Shadows Die Twice,60553783,english,"It's okay, I guess.",1577312423,1577312423,True,10,32,0.572634,...,False,76561198846798934,2159,55,349688.0,0.0,175939.0,1.588410e+09,0.2263,positive
2,Sekiro™: Shadows Die Twice,59813066,english,not bad.,1576305629,1585450858,True,0,2,0.000000,...,False,76561198329996506,149,5,305363.0,0.0,305060.0,1.606347e+09,0.4310,positive
3,Sekiro™: Shadows Die Twice,77197015,english,this is a game about fighting through a supern...,1602102651,1602102651,True,333,19,0.901754,...,False,76561197962008935,278,37,223144.0,0.0,176636.0,1.605052e+09,0.9863,positive
4,Sekiro™: Shadows Die Twice,49702419,english,Yes,1553291428,1560990683,True,2,0,0.545455,...,False,76561198073534796,329,23,113698.0,0.0,2916.0,1.579496e+09,0.4019,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Sekiro™: Shadows Die Twice,78007778,english,Best game uvr,1603444767,1603444767,True,0,0,0.000000,...,False,76561198812769444,50,4,14898.0,53.0,8992.0,1.610004e+09,0.6369,positive
496,Sekiro™: Shadows Die Twice,54494063,english,"It's good and everyone knows it's good, an 8/1...",1565142997,1565142997,True,1,0,0.528986,...,False,76561198042252639,139,20,14887.0,0.0,10227.0,1.604755e+09,-0.3489,negative
497,Sekiro™: Shadows Die Twice,56212472,english,moui epico,1572653706,1572653706,True,0,0,0.000000,...,False,76561198325611644,77,7,14884.0,122.0,2212.0,1.609718e+09,0.0000,neutral
498,Sekiro™: Shadows Die Twice,59311256,english,This gameplay makes darksouls difficulty feel ...,1575588311,1575588311,True,0,1,0.000000,...,False,76561198150429552,79,15,14882.0,0.0,1043.0,1.605605e+09,-0.3470,negative


In [4]:
analyzer = SentimentIntensityAnalyzer()

In [8]:

text = "This is going to be AWSome!"
analyzer.polarity_scores(text)['compound']

# result = 0.0

0.0

In [394]:
add_rating_column(sekiro, 'new_rating')

,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,sentiment,rating,new_rating
0,Sekiro™: Shadows Die Twice,57953979,english,Still can't beat the Blazing Bull. 10/10,1574800184,1574800184,True,1,1,0.520000,...,76561198102339835,2313,23,498718.0,0.0,310344.0,1.590117e+09,0.0000,neutral,neutral
1,Sekiro™: Shadows Die Twice,60553783,english,"It's okay, I guess.",1577312423,1577312423,True,10,32,0.572634,...,76561198846798934,2159,55,349688.0,0.0,175939.0,1.588410e+09,0.2263,positive,positive
2,Sekiro™: Shadows Die Twice,59813066,english,not bad.,1576305629,1585450858,True,0,2,0.000000,...,76561198329996506,149,5,305363.0,0.0,305060.0,1.606347e+09,0.4310,positive,positive
3,Sekiro™: Shadows Die Twice,77197015,english,this is a game about fighting through a supern...,1602102651,1602102651,True,333,19,0.901754,...,76561197962008935,278,37,223144.0,0.0,176636.0,1.605052e+09,0.9863,positive,positive
4,Sekiro™: Shadows Die Twice,49702419,english,Yes,1553291428,1560990683,True,2,0,0.545455,...,76561198073534796,329,23,113698.0,0.0,2916.0,1.579496e+09,0.4019,positive,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Sekiro™: Shadows Die Twice,78007778,english,Best game uvr,1603444767,1603444767,True,0,0,0.000000,...,76561198812769444,50,4,14898.0,53.0,8992.0,1.610004e+09,0.6369,positive,positive
496,Sekiro™: Shadows Die Twice,54494063,english,"It's good and everyone knows it's good, an 8/1...",1565142997,1565142997,True,1,0,0.528986,...,76561198042252639,139,20,14887.0,0.0,10227.0,1.604755e+09,-0.3489,negative,negative
497,Sekiro™: Shadows Die Twice,56212472,english,moui epico,1572653706,1572653706,True,0,0,0.000000,...,76561198325611644,77,7,14884.0,122.0,2212.0,1.609718e+09,0.0000,neutral,neutral
498,Sekiro™: Shadows Die Twice,59311256,english,This gameplay makes darksouls difficulty feel ...,1575588311,1575588311,True,0,1,0.000000,...,76561198150429552,79,15,14882.0,0.0,1043.0,1.605605e+09,-0.3470,negative,negative


In [395]:
sekiro['rating'].value_counts()

rating
positive    362
negative     94
neutral      44
Name: count, dtype: int64

In [396]:
view_game(sekiro[sekiro['rating'] == 'negative'])

,app_name,language,review
5,Sekiro™: Shadows Die Twice,english,10/10 Worst FPS I have ever played
7,Sekiro™: Shadows Die Twice,english,"Hesitate, and you lose.\nso just buy it."
14,Sekiro™: Shadows Die Twice,english,Death\n死\nсмерть\nθάνατος\nDauða\ndød\nטויט\nh...
18,Sekiro™: Shadows Die Twice,english,Sekiro has the most intricate and fast paced c...
19,Sekiro™: Shadows Die Twice,english,What's not to like?
...,...,...,...
487,Sekiro™: Shadows Die Twice,english,Sekiro: Shadows die twice is a game that follo...
489,Sekiro™: Shadows Die Twice,english,A solid if not perfect mix of old and new mech...
494,Sekiro™: Shadows Die Twice,english,This is easily the best game I've played in a ...
496,Sekiro™: Shadows Die Twice,english,"It's good and everyone knows it's good, an 8/1..."


In [397]:
sekiro['review'].iloc[494]

"This is easily the best game I've played in a very long time. It's the sort of game that keeps you coming back for more. That one you think about when your mind should be focused elsewhere. The kind that invades your dreams with the clashing blades. It brings about simply the best satisfaction around when you fell an opponent, not through blind luck, but skill alone. Quick reactions and muscle memory is all that will allow you to proceed. You may find yourself testing the tensile strength of your controller when you get so very close and allow greed to deter you progress seeing but a sliver of health remaining. It will fill you with rage knowing that you didn't succumb to some cheap game mechanic, simply just your own inability, and lack of patience. Despite the grinding teeth, twisted game pad, and numerous Death screens, I found progress from each attempt. Learning the attacks, and how to respond with precision, allowing frustration to gradually be replaced with intent concentration

In [398]:
sekiro['sentiment'].iloc[494]

-0.9309

### PreProcessing

In [399]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Removing Special Characters and Punctuation
    text = re.sub(r'[^\w\s]', ' ', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Handling Numerical Data
    tokens = [token if not token.isdigit() else '__NUM__' for token in tokens]

    # Handling URLs and Email Addresses
    tokens = [token for token in tokens if not re.match(r'^https?://.*|www\..*', token)]
    tokens = [token if '@' not in token else '__EMAIL__' for token in tokens]

    # Stemming (optional)
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Part-of-Speech Tagging and Filtering (keeping only nouns, verbs, adjectives, and adverbs)
    pos_tags = nltk.pos_tag(tokens)
    tokens = [word for word, pos in pos_tags if pos.startswith('N') or pos.startswith('V') or pos.startswith('J') or pos.startswith('R')]

    # Removing Non-Alphanumeric Characters
    tokens = [re.sub(r'[^a-zA-Z]', '', word) for word in tokens]

    # Removing Short Words
    tokens = [word for word in tokens if len(word) > 2]

    return ' '.join(tokens)

In [400]:
sekiro['cleaned_review'] = sekiro['review'].apply(preprocess_text)

In [401]:
sekiro

,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,sentiment,rating,new_rating,cleaned_review
0,Sekiro™: Shadows Die Twice,57953979,english,Still can't beat the Blazing Bull. 10/10,1574800184,1574800184,True,1,1,0.520000,...,2313,23,498718.0,0.0,310344.0,1.590117e+09,0.0000,neutral,neutral,still beat blaze bull num num
1,Sekiro™: Shadows Die Twice,60553783,english,"It's okay, I guess.",1577312423,1577312423,True,10,32,0.572634,...,2159,55,349688.0,0.0,175939.0,1.588410e+09,0.2263,positive,positive,okay guess
2,Sekiro™: Shadows Die Twice,59813066,english,not bad.,1576305629,1585450858,True,0,2,0.000000,...,149,5,305363.0,0.0,305060.0,1.606347e+09,0.4310,positive,positive,bad
3,Sekiro™: Shadows Die Twice,77197015,english,this is a game about fighting through a supern...,1602102651,1602102651,True,333,19,0.901754,...,278,37,223144.0,0.0,176636.0,1.605052e+09,0.9863,positive,positive,game fight supernatur ting recreat centuri jap...
4,Sekiro™: Shadows Die Twice,49702419,english,Yes,1553291428,1560990683,True,2,0,0.545455,...,329,23,113698.0,0.0,2916.0,1.579496e+09,0.4019,positive,positive,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Sekiro™: Shadows Die Twice,78007778,english,Best game uvr,1603444767,1603444767,True,0,0,0.000000,...,50,4,14898.0,53.0,8992.0,1.610004e+09,0.6369,positive,positive,best game uvr
496,Sekiro™: Shadows Die Twice,54494063,english,"It's good and everyone knows it's good, an 8/1...",1565142997,1565142997,True,1,0,0.528986,...,139,20,14887.0,0.0,10227.0,1.604755e+09,-0.3489,negative,negative,good everyon know good num num dark soul rpg p...
497,Sekiro™: Shadows Die Twice,56212472,english,moui epico,1572653706,1572653706,True,0,0,0.000000,...,77,7,14884.0,122.0,2212.0,1.609718e+09,0.0000,neutral,neutral,moui epico
498,Sekiro™: Shadows Die Twice,59311256,english,This gameplay makes darksouls difficulty feel ...,1575588311,1575588311,True,0,1,0.000000,...,79,15,14882.0,0.0,1043.0,1.605605e+09,-0.3470,negative,negative,gameplay make darksoul difficulti feel child l...


In [402]:
add_sentiment_column(sekiro, 'cleaned_review')

,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,sentiment,rating,new_rating,cleaned_review
0,Sekiro™: Shadows Die Twice,57953979,english,Still can't beat the Blazing Bull. 10/10,1574800184,1574800184,True,1,1,0.520000,...,2313,23,498718.0,0.0,310344.0,1.590117e+09,0.0000,neutral,neutral,still beat blaze bull num num
1,Sekiro™: Shadows Die Twice,60553783,english,"It's okay, I guess.",1577312423,1577312423,True,10,32,0.572634,...,2159,55,349688.0,0.0,175939.0,1.588410e+09,0.2263,positive,positive,okay guess
2,Sekiro™: Shadows Die Twice,59813066,english,not bad.,1576305629,1585450858,True,0,2,0.000000,...,149,5,305363.0,0.0,305060.0,1.606347e+09,-0.5423,positive,positive,bad
3,Sekiro™: Shadows Die Twice,77197015,english,this is a game about fighting through a supern...,1602102651,1602102651,True,333,19,0.901754,...,278,37,223144.0,0.0,176636.0,1.605052e+09,0.6369,positive,positive,game fight supernatur ting recreat centuri jap...
4,Sekiro™: Shadows Die Twice,49702419,english,Yes,1553291428,1560990683,True,2,0,0.545455,...,329,23,113698.0,0.0,2916.0,1.579496e+09,0.0000,positive,positive,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Sekiro™: Shadows Die Twice,78007778,english,Best game uvr,1603444767,1603444767,True,0,0,0.000000,...,50,4,14898.0,53.0,8992.0,1.610004e+09,0.6369,positive,positive,best game uvr
496,Sekiro™: Shadows Die Twice,54494063,english,"It's good and everyone knows it's good, an 8/1...",1565142997,1565142997,True,1,0,0.528986,...,139,20,14887.0,0.0,10227.0,1.604755e+09,0.8807,negative,negative,good everyon know good num num dark soul rpg p...
497,Sekiro™: Shadows Die Twice,56212472,english,moui epico,1572653706,1572653706,True,0,0,0.000000,...,77,7,14884.0,122.0,2212.0,1.609718e+09,0.0000,neutral,neutral,moui epico
498,Sekiro™: Shadows Die Twice,59311256,english,This gameplay makes darksouls difficulty feel ...,1575588311,1575588311,True,0,1,0.000000,...,79,15,14882.0,0.0,1043.0,1.605605e+09,0.0516,negative,negative,gameplay make darksoul difficulti feel child l...


In [403]:
add_rating_column(sekiro, 'rating')

,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,...,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,sentiment,rating,new_rating,cleaned_review
0,Sekiro™: Shadows Die Twice,57953979,english,Still can't beat the Blazing Bull. 10/10,1574800184,1574800184,True,1,1,0.520000,...,2313,23,498718.0,0.0,310344.0,1.590117e+09,0.0000,neutral,neutral,still beat blaze bull num num
1,Sekiro™: Shadows Die Twice,60553783,english,"It's okay, I guess.",1577312423,1577312423,True,10,32,0.572634,...,2159,55,349688.0,0.0,175939.0,1.588410e+09,0.2263,positive,positive,okay guess
2,Sekiro™: Shadows Die Twice,59813066,english,not bad.,1576305629,1585450858,True,0,2,0.000000,...,149,5,305363.0,0.0,305060.0,1.606347e+09,-0.5423,negative,positive,bad
3,Sekiro™: Shadows Die Twice,77197015,english,this is a game about fighting through a supern...,1602102651,1602102651,True,333,19,0.901754,...,278,37,223144.0,0.0,176636.0,1.605052e+09,0.6369,positive,positive,game fight supernatur ting recreat centuri jap...
4,Sekiro™: Shadows Die Twice,49702419,english,Yes,1553291428,1560990683,True,2,0,0.545455,...,329,23,113698.0,0.0,2916.0,1.579496e+09,0.0000,neutral,positive,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Sekiro™: Shadows Die Twice,78007778,english,Best game uvr,1603444767,1603444767,True,0,0,0.000000,...,50,4,14898.0,53.0,8992.0,1.610004e+09,0.6369,positive,positive,best game uvr
496,Sekiro™: Shadows Die Twice,54494063,english,"It's good and everyone knows it's good, an 8/1...",1565142997,1565142997,True,1,0,0.528986,...,139,20,14887.0,0.0,10227.0,1.604755e+09,0.8807,positive,negative,good everyon know good num num dark soul rpg p...
497,Sekiro™: Shadows Die Twice,56212472,english,moui epico,1572653706,1572653706,True,0,0,0.000000,...,77,7,14884.0,122.0,2212.0,1.609718e+09,0.0000,neutral,neutral,moui epico
498,Sekiro™: Shadows Die Twice,59311256,english,This gameplay makes darksouls difficulty feel ...,1575588311,1575588311,True,0,1,0.000000,...,79,15,14882.0,0.0,1043.0,1.605605e+09,0.0516,positive,negative,gameplay make darksoul difficulti feel child l...


In [404]:
sekiro['rating'].value_counts()

rating
positive    309
negative    107
neutral      84
Name: count, dtype: int64